In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
import cv2
from random import shuffle
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


In [2]:
from glob import glob
data=list(glob('COVID/*.png'))
print(len(data))
datapng = list(glob('COVID/*.jpg'))
print(len(datapng))
datajpeg=list(glob('COVID/*.jpeg'))
print(len(datajpeg))

514
1796
1734


In [7]:
dataf1=data
dataf1=dataf1+datapng
dataf1=dataf1+datajpeg
print(len(dataf1))



4046


In [5]:
img_rows=64
img_cols=64
images=[]
for i in datajpeg:
    img=cv2.imread(i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray, (img_rows, img_cols))
    images.append(resized_img)



In [6]:
len(images)

4044

In [7]:
images = np.array(images)/255.0
images = np.reshape(images, (images.shape[0], img_rows, img_cols, 1))


In [8]:
images[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.        ],
        [0.07058824],
        [0.12156863],
        ...,
        [0.41176471],
        [0.28627451],
        [0.17647059]],

       [[0.03137255],
        [0.08627451],
        [0.14117647],
        ...,
        [0.43921569],
        [0.29803922],
        [0.18431373]],

       [[0.04705882],
        [0.11372549],
        [0.14117647],
        ...,
        [0.4       ],
        [0.32941176],
        [0.21176471]]])

In [10]:
datan = list(glob('Non-COVID/*.png'))
print(len(datan))
datapngn = list(glob('Non-COVID/*.jpg'))
print(len(datapngn))
datajpegn = list(glob('Non-COVID/*.jpeg'))
print(len(datajpegn))


1861
955
2677


In [13]:
img_rows = 64
img_cols = 64
images1 = []
for i in datan:
    img = cv2.imread(i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray, (img_rows, img_cols))
    images1.append(resized_img)


In [14]:
print(len(images1))
images1 = np.array(images1)/255.0
images1 = np.reshape(images1, (images1.shape[0], img_rows, img_cols, 1))


5493


In [23]:
label=np.empty(4044)
label.fill(1)
#1 means covid and 0 means non
label1=np.empty(5493)
label1.fill(0)

In [24]:
label=np.concatenate((label,label1))
print(label.shape)

(9537,)


In [25]:
label = label.astype(np.int32)

In [18]:
images=np.concatenate((images, images1), axis=0)
images.shape


(9537, 64, 64, 1)

In [26]:
i=images[0:2000]

In [27]:
i1=images[6000:8000]

In [29]:
imagesfinal=np.concatenate((i,i1),axis=0)
imagesfinal.shape

(4000, 64, 64, 1)

In [30]:
labelfinal = np.concatenate((label[0:2000], label[6000:8000]))
labelfinal.shape



(4000,)

In [40]:
lb = LabelBinarizer()
labelfinal = lb.fit_transform(labelfinal)
labelfinal = to_categorical(labelfinal)
labelfinal = np.array(labelfinal)


In [41]:
(X_train, X_test, y_train, y_test) = train_test_split(
    imagesfinal, labelfinal, test_size=0.25, random_state=0)


In [42]:
X_train[0]

array([[[0.12941176],
        [0.09803922],
        [0.09803922],
        ...,
        [0.09803922],
        [0.09803922],
        [0.10196078]],

       [[0.1254902 ],
        [0.09803922],
        [0.09803922],
        ...,
        [0.09411765],
        [0.09803922],
        [0.09411765]],

       [[0.12156863],
        [0.09803922],
        [0.09803922],
        ...,
        [0.09411765],
        [0.09411765],
        [0.09019608]],

       ...,

       [[0.2745098 ],
        [0.36862745],
        [0.41568627],
        ...,
        [0.34901961],
        [0.27058824],
        [0.22352941]],

       [[0.32941176],
        [0.41960784],
        [0.44705882],
        ...,
        [0.35294118],
        [0.29019608],
        [0.23921569]],

       [[0.41960784],
        [0.46666667],
        [0.51372549],
        ...,
        [0.41960784],
        [0.34901961],
        [0.28235294]]])

In [33]:

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [43]:

model = Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                  input_shape=(img_rows, img_cols, 1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])


In [44]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [45]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 16)        9232      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)             

In [46]:
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [47]:
model.fit(X_train, y_train, epochs=50)


Epoch 1/50


C:\Users\HP\Desktop\python\lib\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


94/94 [==============================] - 13s 126ms/step - loss: 0.6385 - accuracy: 0.6330
Epoch 2/50
94/94 [==============================] - 12s 128ms/step - loss: 0.4917 - accuracy: 0.7693
Epoch 3/50
94/94 [==============================] - 12s 123ms/step - loss: 0.4165 - accuracy: 0.8067
Epoch 4/50
94/94 [==============================] - 12s 126ms/step - loss: 0.3572 - accuracy: 0.8437
Epoch 5/50
94/94 [==============================] - 12s 127ms/step - loss: 0.3262 - accuracy: 0.8617
Epoch 6/50
94/94 [==============================] - 12s 130ms/step - loss: 0.2877 - accuracy: 0.8770
Epoch 7/50
94/94 [==============================] - 12s 131ms/step - loss: 0.2695 - accuracy: 0.8900
Epoch 8/50
94/94 [==============================] - 12s 131ms/step - loss: 0.2522 - accuracy: 0.8993
Epoch 9/50
94/94 [==============================] - 12s 131ms/step - loss: 0.2330 - accuracy: 0.9037
Epoch 10/50
94/94 [==============================] - 12s 132ms/step - loss: 0.2184 - accuracy: 0.9153


In [48]:

predictions = model.predict(X_test)


In [50]:
score = (tf.nn.softmax(predictions))


In [54]:
pred=lb.inverse_transform(predictions)
y_te=lb.inverse_transform(y_test)

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te, pred)


0.957